In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
page = requests.get(
    "http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168"
)
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")

In [6]:
tonight = forecast_items[0]
print(tonight.prettify())
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
img = tonight.find("img")
desc = img['title']

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 78. West southwest wind 6 to 15 mph, with gusts as high as 20 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 78. West southwest wind 6 to 15 mph, with gusts as high as 20 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 78 °F
 </p>
</div>


In [7]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday']

In [8]:
# Get descriptions and temperatures
short_descs = [
    sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")
]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

In [9]:
# Creating and extending our dataframe
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": descs
})
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night


In [10]:
# Our full dataframe
weather

,period,short_desc,temp,desc,temp_num,is_night
0,Today,Sunny,High: 78 °F,"Today: Sunny, with a high near 78. West southw...",78,False
1,Tonight,Patchy Fog,Low: 56 °F,"Tonight: Patchy fog after 11pm. Otherwise, in...",56,True
2,Thursday,Patchy Fogthen MostlySunny,High: 70 °F,"Thursday: Patchy fog before 8am. Otherwise, c...",70,False
3,ThursdayNight,IncreasingClouds,Low: 55 °F,"Thursday Night: Increasing clouds, with a low ...",55,True
4,Friday,Partly Sunny,High: 69 °F,"Friday: Partly sunny, with a high near 69. Wes...",69,False
5,FridayNight,Mostly Cloudy,Low: 55 °F,"Friday Night: Mostly cloudy, with a low around...",55,True
6,Saturday,Partly Sunny,High: 68 °F,"Saturday: Partly sunny, with a high near 68.",68,False
7,SaturdayNight,Mostly Cloudy,Low: 55 °F,"Saturday Night: Mostly cloudy, with a low arou...",55,True
8,Sunday,Partly Sunny,High: 71 °F,"Sunday: Partly sunny, with a high near 71.",71,False
